In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('..'))

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import quandl
import cvxportfolio as cp
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

#Quandl financial database user key.
quandl.ApiConfig.api_key = "##########LfHLgiKau2"

In [3]:
#Portfolio stocks.
stocks = ['AKAM', 'JNPR', 'MSI', 'ACN','ADBE','ABT','HCP','BAX','ALXN',
           'PDCO','VIAB','ADS','URBN','ATVI','AMZN','PPL','AEE','NOV','AES','AEP']

#Investment time period.
start_date='2012-01-01'
end_date='2016-12-31'

#Gettind financial data from Quandl database.
Portfolio_returns = pd.DataFrame(dict([(ticker, quandl.get('WIKI/'+ticker,
start_date=start_date,
end_date=end_date)['Adj. Close'].pct_change())
for ticker in stocks]))

#Risk free rate.
Portfolio_returns[["USDOLLAR"]]=quandl.get('FRED/DTB3', 
                                           start_date=start_date, 
                                           end_date=end_date)/(250*100)

Portfolio_returns = Portfolio_returns.fillna(method='ffill').iloc[1:]

In [4]:
#Expected returns and standar deviations of stocks.
historical_data = 250
expected_returns = returns.rolling(window=historical_data, 
                                   min_periods=historical_data).mean().shift(1).dropna()
std_returns = returns.rolling(window=historical_data, 
                              min_periods=historical_data, 
                              closed='neither').cov().dropna()

In [83]:
#Cost models.
Transaction_costs_model=cp.TcostModel(half_spread=10E-4)
Holding_costs_model=cp.HcostModel(borrow_costs=1E-4)

#Risk model.
Risk_model = cp.FullSigma(std_returns)

In [84]:
#Aversion parameters.

#Risk aversion parameters.
RAv1 =1.
RAv2 =5.
RAv3 =10.
RAv4 =20.
RAv5 =50.

#Trading aversion parameters.
TAv1 =0.5
TAv2 =0.7
TAv3 =1.
TAv4 =1.2
TAv5 =1.5

#Holding aversion parameters.
HAv1 =1.
HAv2 =5.
HAv3 =1.
HAv4 =10.
HAv5 =50.

#Leverage limits.
Lev1 = cp.LeverageLimit(1)
Lev2 = cp.LeverageLimit(1.5)
Lev3 = cp.LeverageLimit(2)
Lev4 = cp.LeverageLimit(2.5)
Lev5 = cp.LeverageLimit(3)

In [ ]:
#Investment policies configuration.
MPC_policy1 = cp.MultiPeriodOpt(return_forecast=expected_returns,
                                costs=[RAv1*Risk_model, 
                                       TAv1*Transaction_costs_model, 
                                       HAv1*Holding_costs_model],
                                constraints=[Lev1,cp.LongOnly(),
                                             cp.DollarNeutral()])

MPC_policy2 = cp.MultiPeriodOpt(return_forecast=expected_returns,
                                costs=[RAv2*Risk_model, 
                                       TAv2*Transaction_costs_model, 
                                       HAv2*Holding_costs_model],
                                constraints=[Lev2,cp.LongOnly(),
                                             cp.DollarNeutral()])

MPC_policy3 = cp.MultiPeriodOpt(return_forecast=expected_returns,
                                costs=[RAv3*Risk_model, 
                                       TAv3*Transaction_costs_model, 
                                       HAv3*Holding_costs_model],
                                constraints=[Lev3,cp.LongOnly(),
                                             cp.DollarNeutral()])

MPC_policy4 = cp.MultiPeriodOpt(return_forecast=expected_returns,
                                costs=[RAv4*Risk_model, 
                                       TAv4*Transaction_costs_model, 
                                       HAv4*Holding_costs_model],
                                constraints=[Lev4,cp.LongOnly(),
                                             cp.DollarNeutral()])

MPC_policy5 = cp.MultiPeriodOpt(return_forecast=expected_returns,
                                costs=[RAv5*Risk_model, 
                                       TAv5*Transaction_costs_model, 
                                       HAv5*Holding_costs_model],
                                constraints=[Lev5,cp.LongOnly(),
                                             cp.DollarNeutral()])

In [ ]:
#Market simulation process.
Investment_simulation=cp.MarketSimulator(expected_returns, 
                                         [Transaction_costs_model, 
                                         Holding_costs_model], 
                                         cash_key='USDOLLAR')

#Initial budget: $200,000 ($10,000 per stock).
Initial_portfolio = pd.Series(index=returns.columns, data=10000.)

#Self-funding portfolio.
Initial_portfolio.USDOLLAR = 0

In [ ]:
#Investment simulation process.
Simulation_results = market_sim.run_multiple_backtest(Initial_portfolio,
                                start_time='2013-01-03', end_time='2016-12-29',
                               policies=[cp.Hold(),
                                         MPC_policy1,
                                         MPC_policy2,
                                         MPC_policy3,
                                         MPC_policy4,
                                         MPC_policy5])

In [ ]:
#Display numerical results
Simulation_results[0].summary()
Simulation_results[1].summary()
Simulation_results[2].summary()
Simulation_results[3].summary()
Simulation_results[4].summary()
Simulation_results[5].summary()

In [ ]:
#Display graphical results. Portfolio value over time.
results[0].v.plot(figsize=(15,7),label='Lev=1')
results[1].v.plot(figsize=(15,7),label='Lev=1.5')
results[2].v.plot(figsize=(15,7),label='Lev=2')
results[3].v.plot(figsize=(15,7),label='Lev=2.5')
results[4].v.plot(figsize=(15,7),label='Lev=3')

#Plot configuration
plt.gca().tick_params(axis='both',labelsize=16)
plt.xlabel('Time',fontsize=18)
plt.title('Portfolio value over time',fontsize=18)
plt.axis('tight')
plt.ylabel('Portfolio value ($)',fontsize=18)

ax = plt.gca()
ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, 
                                                     loc: "{:,}".format(int(x))))
plt.legend(loc='upper left',fontsize=18)
plt.savefig('MPC.png')

In [ ]:
#Display graphical results. Portfolio weights over time.
results[0].w.plot(figsize=(15,7),label='Lev=1')
results[1].w.plot(figsize=(15,7),label='Lev=1.5')
results[2].w.plot(figsize=(15,7),label='Lev=2')
results[3].w.plot(figsize=(15,7),label='Lev=2.5')
results[4].w.plot(figsize=(15,7),label='Lev=3')


#Plot configuration
plt.gca().tick_params(axis='both',labelsize=16)
plt.xlabel('Time',fontsize=18)
plt.title('Portfolio weight over time',fontsize=18)
plt.axis('tight')
plt.ylabel('Portfolio weight',fontsize=18)

ax = plt.gca()
ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, 
                                                     loc: "{:,}".format(int(x))))
plt.legend(loc='upper left',fontsize=18)
plt.savefig('MPC2.png')